<a href="https://colab.research.google.com/github/henrykylaw/cakephp-ex/blob/master/Omnizart_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Omniscient Mozart

This is a colab for demonstrating the python package `omnizart` developed by [MCTLab](https://sites.google.com/view/mctl/home).

Github repository can be found in [Music-and-Culture-Technology-Lab/omnizart](https://github.com/Music-and-Culture-Technology-Lab/omnizart).

Official documentation page can be found in [omnizart-doc](https://music-and-culture-technology-lab.github.io/omnizart-doc/)

In [ ]:
#@title Environment Setup

!pip install omnizart
!omnizart download-checkpoints
!apt install fluidsynth
!pip install pyfluidsynth
!curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!chmod a+rx /usr/local/bin/youtube-dl

import os
from google.colab import files
from IPython import display as dsp

# Choose an Audio

Either upload your own MP3 file, or choose from YouTube.


In [ ]:
#@title Upload MP3 File

ff = files.upload()
uploaded_audio = list(ff.keys())[0].replace(".mp3", "")

!test -f "$uploaded_audio".wav && rm "$uploaded_audio".wav
!ffmpeg -i "$uploaded_audio".mp3 "$uploaded_audio".wav &>/dev/null

dsp.Audio(uploaded_audio + ".mp3") if os.path.exists(uploaded_audio + ".mp3") else None

In [ ]:
#@title Choose from YouTube

url = input("Enter your YouTube link: ")

try:
  id = url.split("watch?v=")[1].split("&")[0]
  vid = dsp.YouTubeVideo(id)
  dsp.display(vid)
except Exception:
  pass

print("Downloading...")

!youtube-dl -x --audio-format wav --no-playlist "$url"
!youtube-dl --get-filename --no-playlist "$url" > tmp

uploaded_audio = os.path.splitext(open("tmp").readline().strip())[0]
print(f"Finished: {uploaded_audio}")

# Transcribe the Audio

There are several modes you can choose.
* `music-piano`: transcribe piano solo clips.
* `music-assemble`: transcribe classical assemble pieces.
* `chord`: transcribe chord progressions.
* `drum`: transcribe drum percussion in the audio.

In [ ]:
#@title Transcribe

mode = "music-piano" #@param ["music-piano", "music-assemble", "chord", "drum"]

mode_list = mode.split("-")
mode = mode_list[0]
model = "-".join(mode_list[1:])

from omnizart.music import app as mapp
from omnizart.chord import app as capp
from omnizart.drum import app as dapp

app = {
    "music": mapp,
    "chord": capp,
    "drum": dapp
}[mode]

model_path = {
    "piano": "Piano",
    "assemble": "Stream",
    "pop-song": "Pop",
    "": None
}[model]

midi = app.transcribe(f"{uploaded_audio}.wav", model_path=model_path)

# Synthesize MIDI and play
import scipy.io.wavfile as wave
from omnizart.remote import download_large_file_from_google_drive

SF2_FILE = "general_soundfont.sf2"
if not os.path.exists(SF2_FILE):
  print("Downloading soundfont...")
  download_large_file_from_google_drive(
      "16RM-dWKcNtjpBoo7DFSONpplPEg5ruvO",
      file_length=31277462,
      save_name=SF2_FILE
    )

print("Synthesizing MIDI...")
out_name = f"{uploaded_audio}_synth.wav"
raw_wav = midi.fluidsynth(fs=44100, sf2_path=SF2_FILE)
wave.write(out_name, 44100, raw_wav)

!ffmpeg -i "$out_name" "tmp_synth.mp3" &>/dev/null
!mv tmp_synth.mp3 "$uploaded_audio"_synth.mp3

out_name = out_name.replace(".wav", ".mp3")
print(f"Finished: {out_name}")
dsp.Audio(out_name)

# Download the Transribed MIDI/MP3

In [ ]:
#@title Download MIDI

files.download(f"{uploaded_audio}.mid")

In [ ]:
#@title Download MP3

files.download(out_name)